# Box-counting for 1D sets
## Application to the Fibonacci chain

In [1]:
import numpy as np
from scipy import linalg, stats
import matplotlib.pyplot as plt
import math
from cmath import exp
import time
%matplotlib inline